In [1]:
import polars as pl
import data_actions.data_processing.utils as utils

In [2]:
data = utils.load_data("data/original_data")
utils.calculate_days_since_start(data)

/home/adam/python-projects/recsys25/data_actions/data_processing/utils.py:67: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "timestamp" in df.columns:


In [3]:
days = pl.concat([data["events"][df].select("days_since_start") for df in data["events"]])
max_day = days.max().collect().item()

## Features

Features with which I came up based on the data we have, every feature for three time windows - 1 day, 1 week, 1 month, and for every event among product_buy, add_to_cart, remove_from_cart:
- number of actions
- the three most common item ids (for relevant items)
- the three most common categories (for relevant categories)
- the average or mode price of items to which actions are performed (mode may seem stupid but the prices are given as quartiles)

One additional feature may be to cluster the queries of an user, calculate avg in every cluster and use the most similar item id to the average for every cluster as another feature (We should try to visualize it if it makes any sense before using in training)

For now, I don't have any particular idea for page_visits except:
- number of pages visited in a time window
- the most common page visited

The number of features would be **periods * features * events + clusters + page_visit_features = 3 * 8 * 3 + n + 2 = 74 + n**


In [48]:
event_counts = dict()

for table in ("add_to_cart", "remove_from_cart", "product_buy"):
    event_counts[table] = data["events"][table].group_by(["client_id", "days_since_start"]).agg(
        [
            pl.col("client_id").count().alias(f"count_{table}"),
            pl.col("sku").value_counts(sort=True).struct.field("sku").first().alias(f"most_common_item_{table}"),
        ]
    )


In [54]:
joined_table = list(event_counts.items())[0][1]
for _, table in list(event_counts.items())[1:]:
    joined_table = joined_table.join(table, on=["client_id", "days_since_start"], how="full", coalesce=True)

joined_table = joined_table.fill_null(0)
joined_table.collect()


client_id,days_since_start,count_add_to_cart,most_common_item_add_to_cart,count_remove_from_cart,most_common_item_remove_from_cart,count_product_buy,most_common_item_product_buy
i64,i64,u32,i64,u32,i64,u32,i64
13042918,91,2,993880,0,0,0,0
3982734,134,7,1319275,11,1235754,0,0
21840456,140,5,451894,0,0,0,0
10736527,67,2,537922,1,4262,0,0
22369419,119,1,285430,0,0,0,0
…,…,…,…,…,…,…,…
13188357,74,0,0,0,0,1,130029
12238851,53,0,0,0,0,1,259624
3456497,32,0,0,0,0,3,1424146
